<a href="https://colab.research.google.com/github/LucjanSakowicz/data-science-projects/blob/main/titanic/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [133]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [134]:
df_train[['Surname', 'FirstNames']] = df_train['Name'].str.split(',', expand=True)
df_train[['Surname', 'FirstNames']]

,Surname,FirstNames
0,Braund,Mr. Owen Harris
1,Cumings,Mrs. John Bradley (Florence Briggs Thayer)
2,Heikkinen,Miss. Laina
3,Futrelle,Mrs. Jacques Heath (Lily May Peel)
4,Allen,Mr. William Henry
...,...,...
886,Montvila,Rev. Juozas
887,Graham,Miss. Margaret Edith
888,Johnston,"Miss. Catherine Helen ""Carrie"""
889,Behr,Mr. Karl Howell


In [135]:
df_train[['Title', 'Names', 'Other']] = df_train['FirstNames'].str.split('.', expand=True)
df_train.drop(columns=['FirstNames', 'Name'], inplace=True)

In [136]:
df_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,Title,Names,Other
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,Braund,Mr,Owen Harris,None
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,Cumings,Mrs,John Bradley (Florence Briggs Thayer),None
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Heikkinen,Miss,Laina,None
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S,Futrelle,Mrs,Jacques Heath (Lily May Peel),None
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S,Allen,Mr,William Henry,None


In [137]:
df_train['Other'].value_counts()

 Barrett)    1
Name: Other, dtype: int64

In [138]:
df_train[df_train['Other'].isna() == False]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,Title,Names,Other
513,514,1,1,female,54.0,1,0,PC 17603,59.4,NaN,C,Rothschild,Mrs,Martin (Elizabeth L,Barrett)


In [139]:
df_train.iloc[513, -2] = df_train.iloc[513, -2] + df_train.iloc[513,-1]
df_train.drop(columns='Other', inplace=True)
print(df_train)

     PassengerId  Survived  Pclass     Sex   Age  SibSp  Parch  \
0              1         0       3    male  22.0      1      0   
1              2         1       1  female  38.0      1      0   
2              3         1       3  female  26.0      0      0   
3              4         1       1  female  35.0      1      0   
4              5         0       3    male  35.0      0      0   
..           ...       ...     ...     ...   ...    ...    ...   
886          887         0       2    male  27.0      0      0   
887          888         1       1  female  19.0      0      0   
888          889         0       3  female   NaN      1      2   
889          890         1       1    male  26.0      0      0   
890          891         0       3    male  32.0      0      0   

               Ticket     Fare Cabin Embarked    Surname  Title  \
0           A/5 21171   7.2500   NaN        S     Braund     Mr   
1            PC 17599  71.2833   C85        C    Cumings    Mrs   
2    S

In [140]:
df_train.iloc[513, :]

PassengerId                              514
Survived                                   1
Pclass                                     1
Sex                                   female
Age                                     54.0
SibSp                                      1
Parch                                      0
Ticket                              PC 17603
Fare                                    59.4
Cabin                                    NaN
Embarked                                   C
Surname                           Rothschild
Title                                    Mrs
Names           Martin (Elizabeth L Barrett)
Name: 513, dtype: object

In [151]:
df_train[['Surname', 'Title', 'Names']] = df_train[['Surname', 'Title', 'Names']].apply(lambda x: x.str.strip())

In [152]:
df_train['Surname'].value_counts()

Andersson    9
Sage         7
Panula       6
Skoog        6
Carter       6
            ..
Hanna        1
Lewy         1
Mineff       1
Haas         1
Dooley       1
Name: Surname, Length: 667, dtype: int64

In [158]:
df_train['Names'].value_counts()

John                   9
James                  7
William                6
Mary                   6
Bertha                 4
                      ..
Nora A                 1
Howard Hugh "Harry"    1
Hudson Trevor          1
Margaret               1
Karl Howell            1
Name: Names, Length: 799, dtype: int64

In [160]:
df_train[df_train['Title'] == 'Master']

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,Title,Names
7,8,0,3,male,2.00,3,1,349909,21.0750,NaN,S,Palsson,Master,Gosta Leonard
16,17,0,3,male,2.00,4,1,382652,29.1250,NaN,Q,Rice,Master,Eugene
50,51,0,3,male,7.00,4,1,3101295,39.6875,NaN,S,Panula,Master,Juha Niilo
59,60,0,3,male,11.00,5,2,CA 2144,46.9000,NaN,S,Goodwin,Master,William Frederick
63,64,0,3,male,4.00,3,2,347088,27.9000,NaN,S,Skoog,Master,Harald
65,66,1,3,male,NaN,1,1,2661,15.2458,NaN,C,Moubarek,Master,Gerios
78,79,1,2,male,0.83,0,2,248738,29.0000,NaN,S,Caldwell,Master,Alden Gates
125,126,1,3,male,12.00,1,0,2651,11.2417,NaN,C,Nicola-Yarred,Master,Elias
159,160,0,3,male,NaN,8,2,CA. 2343,69.5500,NaN,S,Sage,Master,Thomas Henry
164,165,0,3,male,1.00,4,1,3101295,39.6875,NaN,S,Panula,Master,Eino Viljami
